# Imports necessários

In [1]:
# IGNORAR
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Pacotes
nltk.download('stopwords')
nltk.download('punkt')
nlp = spacy.load('pt_core_news_sm')
stop_words = set(stopwords.words('portuguese'))

# Tokenizador e modelo do BERT
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('./best_models/top3')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Definição das funções

Preparo do Corpus

In [3]:
# Limpeza do texto
def text_cleaning(text):
    text = re.sub(r'@\w+|USER', '', text) # Usernames
    text = re.sub(r'http\S+', '', text) # Links
    text = re.sub(r'\brt\b|\bRT\b', '', text) # RTs
    text = re.sub(r'#\w+', '', text) # Hashtags
    text = re.sub(r'\n', '', text) # Quebras de linha
    text = re.sub(r'\w*\d\w*', '', text) # Palavraas com dígitos
    text = re.sub(r'\b\w{1,2}\b', '', text) # Palavras com 1 ou 2 caracteres
    text = re.sub(r'[^\w\s]', ' ', text) # Pontuações e caracteres especiais
    text = re.sub(r'\s+', ' ', text) # Espaços extras
    text = text.lower() # Normalização
    text = ' '.join(word for word in text.split() if word not in stop_words) # Stop Words
    return text

# Tokenização
def text_tokenizing(text):
    text = word_tokenize(text) # Tokenização
    text = [token.lemma_ for token in nlp(' '.join(text))] # Lematização
    text = ' '.join(text) # Junção em uma string
    return text

Processamento da entrada e definição do modelo

In [4]:
# Processamento da entrada
def process_text(text):
    text = text_cleaning(text) # Limpeza
    text = text_tokenizing(text) # Tokenização
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt") # Tokenização do BERT
    input_ids = torch.tensor(inputs['input_ids'])
    attention_mask = torch.tensor(inputs['attention_mask'])
    return input_ids, attention_mask

def create_table(probabilities):
    categories = ['homophobia', 'obscene', 'insult', 'racism', 'sexism', 'xenophobia']
    table = pd.DataFrame({'Confidence': probabilities}, index=categories)
    return table

def run_model(text):
    input_ids, attention_mask = process_text(text)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)   
    predictions = outputs.logits
    probabilities = torch.sigmoid(predictions)
    table = create_table(probabilities[0].tolist())
    return table

# Testes

In [5]:
# Tweets de teste
test_0 = 'Bom dia, meu nome é Pedro, estou testando aqui, espero que dê certo.'
test_h = 'Vai se fuder, seu viado'
test_o = 'Chupa meu penis'
test_i = 'Você é feio e burro'
test_r = 'Negro é tudo macaco'
test_s = 'Lugar de mulher é na cozinha'
test_x = 'Nordestinos estúpidos, odeio esses paraíbas'

In [6]:
# Caso de teste não ofensivo
print(test_0, '\n', '\n', run_model(test_0))

Bom dia, meu nome é Pedro, estou testando aqui, espero que dê certo. 
 
             Confidence
homophobia    0.003036
obscene       0.032702
insult        0.029382
racism        0.004499
sexism        0.006886
xenophobia    0.005788


In [7]:
# Caso de teste homofobia
print(test_h, '\n', '\n', run_model(test_h))

Vai se fuder, seu viado 
 
             Confidence
homophobia    0.810573
obscene       0.710732
insult        0.726098
racism        0.067573
sexism        0.114067
xenophobia    0.037959


In [8]:
# Caso de teste obscenidade
print(test_o, '\n', '\n', run_model(test_o))

Chupa meu penis 
 
             Confidence
homophobia    0.035690
obscene       0.943469
insult        0.226441
racism        0.027718
sexism        0.210055
xenophobia    0.011054


In [9]:
# Caso de teste insulto
print(test_i, '\n', '\n', run_model(test_i))

Você é feio e burro 
 
             Confidence
homophobia    0.008164
obscene       0.034265
insult        0.878321
racism        0.012569
sexism        0.013404
xenophobia    0.042368


In [10]:
# Caso de teste racismo
print(test_r, '\n', '\n', run_model(test_r))

Negro é tudo macaco 
 
             Confidence
homophobia    0.159203
obscene       0.041068
insult        0.569003
racism        0.060268
sexism        0.051079
xenophobia    0.142132


In [11]:
# Caso de teste sexismo
print(test_s, '\n', '\n', run_model(test_s))

Lugar de mulher é na cozinha 
 
             Confidence
homophobia    0.043930
obscene       0.191062
insult        0.598940
racism        0.040935
sexism        0.589884
xenophobia    0.038734


In [12]:
# Caso de teste xenofobia
print(test_x, '\n', '\n', run_model(test_x))

Nordestinos estúpidos, odeio esses paraíbas 
 
             Confidence
homophobia    0.008521
obscene       0.035702
insult        0.411464
racism        0.018352
sexism        0.041906
xenophobia    0.068300
